<a href="https://colab.research.google.com/github/Alecia113/NLP-Lab/blob/main/lab2_learnbyself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Word2Vec**

In [11]:
#开始import
import pprint
import re

from lxml import etree  # for parsing Our XML data  用于解析我们的XML数据。
'''
数据存储数据分析
可扩展标记语言（标准通用标记语言的子集）是一种简单的数据存储语言。
XML文件一般指里面写有可扩展标记语言的文件。 XML:可扩展标记语言,标准通用标记语言的子集,是一种用于标记电子文件使其具有结构性的标记语言。它被设计用来传输和存储数据。 XML,是Extensible Markup Language 的缩写。在.NET框架中XML是非常重要的一部分...”
'''
import nltk  # data precessing
nltk.download('punkt')  #很固定
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec  # implementing the word2vec family of algorithms.
#用于实现word2vec 系列算法 ms。

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
#https://www.cnblogs.com/leomei91/p/7660928.html
#https://blog.csdn.net/weixin_42214743/article/details/107720074
#有些警告是可以忽略的
'''
调用模型还是调整参数

import warnings
warnings.filterwarnings('ignore')

具体函数：
warnings.filterwarnings(action, message='', category=Warning, module='', lineno=0, append=False)
'''

"\n调用模型还是调整参数\n\nimport warnings\nwarnings.filterwarnings('ignore')\n\n具体函数：\nwarnings.filterwarnings(action, message='', category=Warning, module='', lineno=0, append=False)\n"

**然后开始下载数据** use TED script(脚本） from google drive

In [13]:
# 下面代码是： generate a link +  a field (字段) ---> verification code (enter)
#   click link  go to google sign in ; ; (own google account) 
#  copy  verification code + into field + press enter
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth  #GoogleAuthenticator(谷歌身份验证器)
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials  #谷歌  全权证书

#Authenticate  [ɔːˈθentɪkeɪt]  v  鉴定，验证（凹三忒忒咳）
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

下载TED Scripts from GD

In [14]:
#点击files tab + see file is download successfully
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('ted_en-20160408.xml')

#在路径中就会过个/content/ted_en-20160408.xml

**数据处理 data preprocessing**

In [18]:
targetXML =open('ted_en-20160408.xml','r',encoding = 'UTF8') #等号后面必须要直接跟open；用utff8来打开ted文件
#open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)

target_text = etree.parse(targetXML)#etree.parse直接接受一个文档，按照文档结构解析（本地文件）
'''
etree.parse(*args, **kwargs)
parse(source, parser=None, base_url=None)
'''
#etree.html可以解析html文件：（服务器上返回的html数据）
#https://blog.csdn.net/cbiexi/article/details/104479744?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161568688316780255255960%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161568688316780255255960&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-3-104479744.first_rank_v2_pc_rank_v29&utm_term=etree.parse
#https://blog.csdn.net/myself029/article/details/79954301?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161568688316780255255960%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161568688316780255255960&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-2-79954301.first_rank_v2_pc_rank_v29&utm_term=etree.parse
#etree 更像是展示出来这个文本文档等等
parse_text = '\n'.join(target_text.xpath('//content/text()')) #获取<内容>标记的内容从xml 文件中【par解析文字】
# 从xml文件中获取<content>标签的内容。
# 【在爬取内容】"\n".join() 来对字符串列表进行处理，不会出现不连贯情况
#https://blog.csdn.net/freeking101/article/details/64461574?utm_medium=distribute.pc_relevant.none-task-blog-OPENSEARCH-1.control&dist_request_id=&depth_1-utm_source=distribute.pc_relevant.none-task-blog-OPENSEARCH-1.control
#https://blog.csdn.net/weixin_39751871/article/details/111537728?ops_request_misc=&request_id=&biz_id=102&utm_term=%27%5Cn%27.join(target_text.xpath(%27/&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-0-111537728.first_rank_v2_pc_rank_v29


content_text = re.sub(r'\([^)]*\)','',parse_text) # 删除音效标签（音频，笑声）#获得文本
'''
\: 将下一个字符————》特殊字符、原义字符。[eg]:\( ==(
^ : 匹配输入字符串的开始位置
*: 匹配前面的子表达式0次或者多次 * == {0,}



'''
#regex 检索词、、正则表达式
#正则表达式强大的文本匹配功能。很多文本匹配处理，如果没有正则表达式，还真的很难做出来。
#https://blog.csdn.net/wyb880501/article/details/79813658

sent_text = sent_tokenize(content_text) #利用NLTK库 对句子进行标记化处理。# 将句子分开。sent_tokenize
#https://blog.csdn.net/wyb880501/article/details/79813658
#regex(蕊·寨·课·嘶)n
#https://blog.csdn.net/Wisimer/article/details/89556404?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161569572316780271589227%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161569572316780271589227&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_click~default-1-89556404.first_rank_v2_pc_rank_v29&utm_term=NLTK
#NLTK 的讲解  

normalized_text = []     #归一化文本  normalized (孬·某·赖·子·的)
for string in sent_text:  #从你的语句中获得单词
  tokens = re.sub


In [22]:
content_text[:1000] #个数；是字母的个数（获取文本）

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit. I'm actually old enough to remember them. Facit was a fantastic company. They were born deep in the Swedish forest, and they made the best mechanical calculators in the world. Everybody used them. And what did Facit do when the electronic calculator came along? They continued doing exactly the same. In six months, they went from maximum revenue ... and they were gone. Gone.\nTo me, the irony about the Facit story is hearing about the Facit engineers, who had bought cheap, small electronic calculators in Japan that they used to double-check their calculators.\n\nFacit did too much exploitation. But exploration can go wild, too.\nA few years back, I worked closely alongside a European bio

In [25]:
sent_text[:10] #十句话。

["Here are two reasons companies fail: they only do more of the same, or they only do what's new.",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation.',
 'Both are necessary, but it can be too much of a good thing.',
 'Consider Facit.',
 "I'm actually old enough to remember them.",
 'Facit was a fantastic company.',
 'They were born deep in the Swedish forest, and they made the best mechanical calculators in the world.',
 'Everybody used them.',
 'And what did Facit do when the electronic calculator came along?',
 'They continued doing exactly the same.']